In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver import DesiredCapabilities
import time

In [2]:
url = "https://scorecard.lcv.org/members-of-congress"

In [3]:
options = webdriver.ChromeOptions()

options.add_argument('--disable-web-security')
options.add_argument('--ignore-certificate-errors') 
options.add_argument('--allow-running-insecure-content')

capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True 
capabilities['acceptInsecureCerts'] = True

In [4]:
driver = webdriver.Chrome(executable_path = '/Users/claytonaldern/Desktop/chromedriver',
                          options=options,
                          desired_capabilities=capabilities)

In [5]:
driver.implicitly_wait(30)
driver.get(url)
yearsElement = driver.find_element_by_id("moc-list-year-selector")
options = [x for x in yearsElement.find_elements_by_tag_name("option")]
yearSelect = Select(yearsElement)

d = pd.DataFrame(columns=['Representative',
                          'Party',
                          'District',
                          'Score',
                          'Year',
                          'Lifetime Score'])

In [6]:
for o in options:
    year = o.get_attribute("value")
    yearSelect.select_by_value(year)
    time.sleep(5)
    
    html = driver.page_source
    search_soup = BeautifulSoup(html, 'html.parser')
    tbl = search_soup.find(id="moc-list-table-data").findAll(class_='tableRow')
    
    t = pd.DataFrame(columns=['Representative',
                          'Party',
                          'District',
                          'Score',
                          'Year',
                          'Lifetime Score'])
    r = 0

    for rep in tbl:
        t.loc[r,'Representative'] = rep.find(class_='mocName').text
        t.loc[r,'Party'] = rep.find(class_='mocParty').text
        t.loc[r,'District'] = rep.find(class_='mocState').text
        t.loc[r,'Score'] = rep.find(class_='mocRating').text
        t.loc[r,'Year'] = year
        t.loc[r,'Lifetime Score'] = rep.find(class_='mocLifetimeRating').text
        r += 1
        
    d = d.append(t, ignore_index=True)

In [7]:
d

,Representative,Party,District,Score,Year,Lifetime Score
0,"Jones, Doug",D,AL,86%,2019,82%
1,"Shelby, Richard C.",R,AL,29%,2019,13%
2,"Byrne, Bradley",R,AL-01,3%,2019,0%
3,"Roby, Martha",R,AL-02,3%,2019,4%
4,"Rogers, Mike D.",R,AL-03,3%,2019,5%
...,...,...,...,...,...,...
26270,"Davis, Glenn Robert",R,WI-09,27%,1971,14%
26271,"O'Konski, Alvin Edward",R,WI-10,40%,1971,39%
26272,"McGee, Gale William",D,WY,38%,1971,28%
26273,"Hansen, Clifford Peter",R,WY,13%,1971,11%


In [8]:
d['Score'] = d['Score'].str.replace('%','')
d['Score'] = pd.to_numeric(d['Score'], errors='coerce') / 100
d['Lifetime Score'] = d['Lifetime Score'].str.replace('%','')
d['Lifetime Score'] = pd.to_numeric(d['Lifetime Score'], errors='coerce') / 100
d

,Representative,Party,District,Score,Year,Lifetime Score
0,"Jones, Doug",D,AL,0.86,2019,0.82
1,"Shelby, Richard C.",R,AL,0.29,2019,0.13
2,"Byrne, Bradley",R,AL-01,0.03,2019,0.00
3,"Roby, Martha",R,AL-02,0.03,2019,0.04
4,"Rogers, Mike D.",R,AL-03,0.03,2019,0.05
...,...,...,...,...,...,...
26270,"Davis, Glenn Robert",R,WI-09,0.27,1971,0.14
26271,"O'Konski, Alvin Edward",R,WI-10,0.40,1971,0.39
26272,"McGee, Gale William",D,WY,0.38,1971,0.28
26273,"Hansen, Clifford Peter",R,WY,0.13,1971,0.11


In [9]:
d.to_csv(r'lcv.csv', index = False)

In [10]:
driver.quit()